# Formalia:

Please read the [assignment overview page](https://github.com/suneman/socialgraphs2025/wiki/Assignments) carefully before proceeding. This page contains information about formatting (including formats etc), group sizes, and many other aspects of handing in the assignment. 

_If you fail to follow these simple instructions, it will negatively impact your grade!_

**Due date and time**: The assignment is due on Tuesday September 30th, 2025 at 23:55. Hand in your IPython notebook file (with extension `.ipynb`) via DTU Learn

# Assignment 1.1: Exploring WS and BA models

This first part draws on the Watts-Stogatz and Barabasi-Albert models from Week 3. You should provide solutions to the exercises with the following titles from **Part 1** 

* *Did you really read the text? Answer the following questions (no calculations needed) in your IPython notebook*

* *WS edition*

And from **Part 2**

* *BA Edition*.
  * **Note**: The second part of this exercise (after the questions to the text) first has you build a BA network step-by-step, but doesn't ask any questions. For that part, I would simply like you to write well-documented code that shows how you build the network. 


# Assignment 1.2: Stats and visualization of the Rock Music Network

This second part requires you to have built the network of Rock Musicians as described in the exercises for Week 4. You should complete the following exercise from **Part 2**.

* *Explain your process in words*

* *Simple network statistics and analysis*.

  * **Note related to this and the following exercise**. It is nice to have the dataset underlying the statistics and visualization available when we grade. Therefore, I recommend that you create a small *network dataset*, which is simply your graph stored in some format that you like (since it's only a few hundred nodes and a few thousand edges, it won't take up a lot of space). You can then place that network one of your group members' GitHub account (or some other server that's available online) and have your Jupyter Notebook fetch that dataset when it runs. (It's OK to use an LLM for help with setting this up, if it seems difficult). 

And the following exercise from **Part 3**

* *Let's build a simple visualization of the network*

And that's it! You're all set.

### 1.2 PART 2 - Explain process in words

The first thing that we did was to trying to understand how Wikipedia stored links inside pages and we played a bit for understanding how to extract single performer names from the List of mainstream rock performers provided using RegEx.

When we understood how to do it, with a function we extracted all the names of the page, ending up with 490 titles, with also a sanity check for veryfing that any other title (for examples reference links) ended up in our list.

Then we have set up a function that, for every performer title found, call the endpoint of Wikipedia API in order to get the cleanest source text for each performer's page (link-parking would have been easier).
We downloaded as wikitext since keeps internal links perfectly for then parsing with RegEx and saved as one .txt per title.

Then, at this point, we had all the elements needed for building the network: each performer bacame a not and every performer page has been parsed with a regex to extract outgoing kinks; if a link pointed to another performer in my set we added a directed edge A->B. Then we computed a word count from the cleaned wikitext and stored it as a content_length attribute on each node. This can be used as a metric for how much information exists about a performer and and we hoped then to can correlate this to the number of outgoing/ingoing edges.

Finally we removed isolated nodes (no in- or out-links), then kept the largest weakly connected component (LWCC) for analysis.

The hardest part was to correctly extract and filter the links. Wikipedia pages are full of internal references, so it was still necessary to ignore links belonging to other namespaces such as Category: or File:, and to double-check that the links we kept really matched one of our performers. 
We used LLM mostly as a way to create the pipeline and to test whether regexes and API calls were set up in a robust way. Once I had these suggestions, I always verified them manually on a few performer pages, adjusted when needed, and only then scaled up the process.

We don't really know what we would change if we would need to do it again, maybe inspecting other types of parser which would make link extraction and text cleaning more robust. 

### PART 2 - Simple network statistics and analysis